## gpt-3.5-turbo 모델 (RAG X)

In [6]:
from openai import OpenAI
from getpass import getpass
import json
import re
from tqdm import tqdm

In [8]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [7]:
with open('data/check_card_info-ver-4.json', 'r', encoding='utf-8') as json_file:
    check_card_data = json.load(json_file)

with open('data/credit_card_info-ver-4.json', 'r', encoding='utf-8') as json_file:
    credit_card_data = json.load(json_file)

In [4]:
user_content = "28세 사회 초년생으로, 매일 출근 전 오전 8시에 스타벅스에서 커피를 사며, 출근 할 때 OTT를 주로 시청하며 이동한다. 주말에는 쇼핑이 주된 취미이며, 1달에 2박3일정도 해외여행을 다닌다."

#### ※ gpt-3.5-model로 15번의 응답 저장(RAG X) : 프롬프트 고도화만 진행

In [2]:
base_template = f'''다음은 카드 추천 양식입니다.
    세부적인 설명이 없을 때는 데이터의 가장 첫번째 카드(1위)로 추천합니다.

    카드의 추천 양식은 아래의 양식을 참고하십시오.

    1. 추천카드의 이름, 혜택요약, 연회비 순서로 작성하십시오.
        1-1. 주요 혜택이 사용자의 어떤 습관과 연관이 있는지 순서 쌍으로 작성하십시오.
        1-2. 주요 혜택 중 연관이 없는 나머지 혜택에 대해 작성하십시오.
        
    1번의 예시입니다.
    - 추천 카드 : **KB국민 My WE:SH 카드**
    - 혜택 요약 : KB Pay 10% 할인, 음식점 10% 할인, 서비스팩 3개 중 1개 선택할인
    - 연회비: 국내전용 15,000원 - 모바일단독카드 9,000원, 해외겸용 15,000원 - 모바일단독카드 9,000원
    
    - 주요 혜택(매칭) :
            음식점 10% 할인) 사용자가 점심을 회사 근처 맛집에서 자주 먹기 때문에 음식점 할인 혜택이 있는 카드가 유리합니다.
            이동수단 10% 할인) 사용자가 주 5일 버스로 회사를 출퇴근하기 때문에 이동수단에 할인이 있는 카드를 추천합니다.

    - 매칭되지 않은 혜택: KB Pay 10% 할인, 서비스팩 3개 중 1개 선택할인 

    2. 다음으로 카드의 전반적인 평가에 대해 작성하십시오.
        2-1. 추천 카드의 혜택 이외에 고려할만한 혜택에 대해 추가 작성하십시오.

    2번의 예시입니다.

    이 카드는 음식점 할인, 통신요금 할인 및 디지털 구독 서비스 할인을 제공하여 사용자의 생활 습관과 잘 맞습니다. 또한, 해외 결제에 대한 혜택이 포함되어 있어 해외 웹툰 구매에도 유리할 수 있습니다.
    추가로 이 사용자의 생활 습관과 필요에 가장 적합한 신용카드를 추천하기 위해 다음과 같은 혜택을 고려할 수 있습니다:

    1) **주유 할인 또는 적립**: 자차를 이용해 출퇴근하고 주말 드라이브를 즐기므로 주유 할인이나 적립 혜택이 있는 카드가 적합합니다.
    2) **해외 웹툰 구매 혜택**: 해외 웹툰 구매를 즐기므로 해외 결제에 대한 할인이나 적립 혜택이 있는 카드가 좋습니다.
    3) **건강 및 웰니스 혜택**: 요가와 필라테스를 주기적으로 하며 유기농 식재료를 사용하므로 건강 및 웰니스 관련 혜택이 있는 카드가 유용합니다.
    4) **여행 혜택**: 부모님과의 여행 계획이 있으므로 여행 관련 혜택이 포함된 카드가 도움이 될 수 있습니다.

    ---------------------------------------------------------------------------------------------------------------------------------------
    ※※**추천 작성 시 주의사항** : 
    1. 주요 혜택 매칭을 할 때, 사용자의 습관의 카테고리와 주요 혜택과 연관된 걸 우선적으로 보고, 주요 혜택의 세부 설명에 연관된 걸 찾아줘

    안좋은 예시) 건강식을 자주 만들어 가족과 함께 식사를 즐기는 사용자에게는 마일리지 적립 혜택이 유용합니다.
    건강식이랑 마일리지는 직접적인 연관이 없습니다.

    2. 사용자 소비 패턴에 따른 논리적 연결 강화
    단순히 특정 카테고리를 소비한다고 해서 무조건 그 혜택을 추천하는 것이 아니라, 소비 동기와 실제 혜택 활용 가능성을 고려해야 합니다.

    "사용자의 소비 패턴과 혜택의 실제 활용 가능성을 고려하여 추천하라."
    "사용자가 특정 카테고리를 자주 소비하더라도, 그 소비 방식(예: 온라인/오프라인, 정기적/비정기적)이 혜택과 적절히 매칭되는지 검토하라."

    💡 예시 적용:

    "자택에서 식사하며 음식 재료를 온라인으로 구매하는 사용자는 음식점 할인보다 온라인 마트/배달 할인 혜택이 적합하다."
    "대도시 외곽에서 출퇴근하는 사용자는 대중교통 이용이 많다면 교통비 할인, 자가용 이용이 많다면 주유 할인 혜택이 유용하다."

    **그리고 카드 데이터의 없는 헤택을 만들어내지 마십시오. 추천할만한 카드가 없으면 그나마 비슷한 혜택의 카드를 가져와주세요**'''

In [229]:
def recommend_my_model(check_card_data, credit_card_data, recommend_template, user_content):
    # 체크카드와 신용카드를 한번에 넣게 되면 토큰 수가 초과되기 때문에 if-elif-else 문으로 종류별 구분하여 실행
    # 체크카드와 신용카드의 구분 가이드
    content = f'''당신은 소비 패턴을 분석하는 카드 추천 전문가입니다. 사용자의 소비 습관에 맞춰 가장 효율적인 카드를 추천합니다. 
    질문을 고려하여 체크카드 또는 신용카드를 추천해야 하며, 
    신용카드의 추천기준으로는

    **소득 안정성**: 신용카드는 신용도에 따라 한도가 설정되기 때문에, 정기적이고 안정적인 소득을 가진 사람이 적합합니다. 예를 들어, 월급을 받는 직장인이나 사업자가 이에 해당합니다.
    포인트/혜택 활용: 다양한 포인트 적립, 캐시백, 할인 혜택을 활용할 수 있습니다. 예를 들어, 자주 쇼핑을 하거나 여행을 즐기는 경우, 해당 분야에 특화된 혜택을 제공하는 카드를 추천할 수 있습니다.
    지출 관리 능력: 신용카드는 일정 기간 동안 결제하고 나중에 상환하는 방식이므로, 지출 관리 능력이 중요한 요소입니다. 월별로 계획적인 예산을 세우고, 적시에 결제할 수 있는 능력이 있는 사람에게 적합합니다.
    일시불/분할결제 선호도: 일시불 결제 또는 분할 결제 혜택을 선호하는 경우, 해당 조건을 잘 반영한 카드를 추천해야 합니다. 예를 들어, 고급 소비 성향이나 큰 금액의 지출을 분할 상환할 필요가 있는 경우 유리합니다.

    체크카드의 추천기준으로는

    **재정 관리 중요성**: 체크카드는 계좌에 있는 한도 내에서만 결제가 가능하므로, 소비를 통제하고 싶은 사람에게 적합합니다. 신용카드에 비해 즉시 결제되기 때문에 과도한 지출을 피하고 싶은 사람에게 유리합니다.
    매일 자주 사용하는 카드: 일상적인 소비에서 작고 빈번한 지출이 많은 사람에게 적합합니다. 예를 들어, 카페, 편의점, 대중교통 등에서 자주 결제하는 소비 성향이라면 체크카드가 적합할 수 있습니다.
    상환 부담을 피하고 싶은 사람: 신용카드는 대금 결제 후 일시불이나 할부 상환 방식을 선택해야 하지만, 체크카드는 즉시 결제되므로 상환에 대한 부담이 없습니다. 이로 인해 상환 계획 없이 즉시 결제가 가능한 카드를 원하는 소비자에게 유리합니다.
    간편한 관리 선호자: 체크카드는 신용카드에 비해 관리가 간편하고, 과소비를 방지하기 위해 필요한 사람에게 적합합니다.
    직업이 없는 사람(무직, 백수), 가진 재산이 부족한 사람은 체크카드가 적합합니다.

    기타 고려 사항으로
    **연회비 부담**: 신용카드는 카드사마다 연회비가 다르기 때문에 연회비가 부담스러운 사람은 혜택이 좋은 체크카드를 추천할 수 있습니다.
    적립 혜택 및 할인율: 신용카드는 다양한 적립 프로그램과 할인 혜택이 있지만, 혜택을 자주 활용하지 않는 소비자에게는 체감 혜택이 적을 수 있으므로 이 점도 고려해야 합니다.

    **답변은 단답으로 정확히 체크카드 아니면 신용카드 로만 대답하십시오.**'''


    # 체크카드 추천 챗봇의 페르소나
    check_content = f'''당신은 체크카드 추천 전문가입니다. 사용자의 소비 습관에 맞춰 가장 효율적인 카드를 추천합니다.
    다음 주어지는 체크카드 파일 중에서 추천해야 합니다.
    체크카드 : {check_card_data}
    작성 양식 : {recommend_template}'''


    # 신용카드 추천 챗봇의 페르소나
    credit_content = f'''당신은 신용카드 추천 전문가입니다. 사용자의 습관에 맞춰 가장 효율적인 카드를 추천합니다.
    다음 주어지는 신용카드 파일 중에서 추천해야 합니다.
    신용카드 : {credit_card_data}
    작성 양식 : {recommend_template}'''


    # 유저의 페르소나를 탐색하여 먼저 체크카드를 추천해야할지, 신용카드를 추천해야할지 선정
    choice = client.chat.completions.create(model='gpt-3.5-turbo',
                                                messages=[{'role':'system','content':content},
                                                        {'role':'user','content':user_content}],
                                                        temperature=0)

    # 신용카드 또는 체크카드를 what_card에 저장
    what_card = choice.choices[0].message.content

    # what_card의 결과에 따라 체크 or 신용 카드를 추천
    if what_card == '체크카드':
        completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                                messages=[{'role':'system','content':check_content},
                                                        {'role':'user','content':user_content}],
                                                        temperature=0)
    elif what_card == '신용카드':
        completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                                messages=[{'role':'system','content':credit_content},
                                                        {'role':'user','content':user_content}],
                                                        temperature=0)

    return completion.choices[0].message.content

In [1]:
# pairwise_eval에 넣을 15개의 답변
answer_a = [recommend_my_model(check_card_data, credit_card_data, base_template, user_content) for _ in tqdm(range(15))]

NameError: name 'tqdm' is not defined

In [132]:
print(answer_a[10])

**추천 카드 : American Express Blue**

- 혜택 요약 : 해외 5% 적립, 교통·통신 5% 적립, 편의점·배달앱 7% 적립
- 연회비 : 해외겸용 15,000원

- 주요 혜택(매칭) :
    - **카페**: 스타벅스·이디야 20% 결제일 할인
    - **디지털구독**: 스트리밍서비스 50% 결제일 할인

- 매칭되지 않은 혜택: 쇼핑 1.5% 멤버십리워즈 적립

**전반적인 평가**:
이 카드는 스타벅스에서의 커피 구매와 스트리밍 서비스 이용에 대한 할인 혜택을 제공하여 사용자의 일상 생활과 잘 맞습니다. 또한, 해외에서의 쇼핑과 여행에 대한 혜택도 있어 주말 쇼핑과 해외여행에 유용할 것입니다. 추가로, 해외에서의 이용에 대한 5% 멤버십리워즈 적립 혜택이 있어 해외여행 시 추가 혜택을 누릴 수 있습니다.


#### ※ GPT-4o 모델의 결과 똑같은 텍스트로 15번 받아가기

In [ ]:
content = f'''사용자의 소비 습관에 맞춰 가장 효율적인 카드를 추천합니다. 
질문을 고려하여 체크카드 또는 신용카드를 추천해야 합니다.

**답변은 단답으로 정확히 체크카드 아니면 신용카드 로만 대답하십시오.**'''

check_content = f'''사용자의 소비 습관에 맞춰 가장 효율적인 카드 1개를 추천합니다.
다음 주어지는 체크카드 파일 중에서 추천해야 합니다.
체크카드 : {check_card_data}'''

credit_content = f'''사용자의 습관에 맞춰 가장 효율적인 카드 1개를 추천합니다.
다음 주어지는 신용카드 파일 중에서 추천해야 합니다.
신용카드 : {credit_card_data}'''


# 카드 추천을 받을 유저 페르소나
user_content = '''28세 사회 초년생으로, 매일 출근 전 오전 8시에 스타벅스에서 커피를 사며, 출근 할 때 OTT서비스를 주로 시청하며 이동한다. 주말에는 쇼핑이 주된 취미이며, 1달에 2박3일정도 해외여행을 다닌다.
'''

# 유저의 페르소나를 탐색하여 먼저 체크카드를 추천해야할지, 신용카드를 추천해야할지 선정
choice = client.chat.completions.create(model='gpt-4o',
                                            messages=[{'role':'system','content':content},
                                                      {'role':'user','content':user_content}],
                                                      temperature=0)

# 신용카드 또는 체크카드를 what_card에 저장
what_card = choice.choices[0].message.content
print(f'{what_card}')

# what_card의 결과에 따라 체크 or 신용 카드를 추천
if what_card == '체크카드':
    completion = client.chat.completions.create(model='gpt-4o',
                                            messages=[{'role':'system','content':check_content},
                                                      {'role':'user','content':user_content}],
                                                      temperature=0)
elif what_card == '신용카드':
    completion = client.chat.completions.create(model='gpt-4o',
                                            messages=[{'role':'system','content':credit_content},
                                                      {'role':'user','content':user_content}],
                                                      temperature=0)

completion.choices[0].message.content

신용카드


"사용자의 소비 습관에 맞춰 가장 효율적인 카드는 'American Express Blue' 카드입니다. 이 카드는 다음과 같은 혜택을 제공합니다:\n\n- 스타벅스와 이디야에서 20% 결제일 할인을 받을 수 있어 매일 아침 커피 구매에 유리합니다.\n- 스트리밍 서비스에 대해 20% 결제일 할인을 제공하여 출근 시 스트리밍 시청에 적합합니다.\n- 해외 이용 시 5% 멤버십 리워즈 적립이 가능하여 해외여행 시에도 혜택을 받을 수 있습니다.\n- 쇼핑 시 1.5% 멤버십 리워즈 적립이 가능하여 주말 쇼핑 취미에도 적합합니다.\n\n이 카드는 사용자의 주요 소비 패턴에 맞춰 다양한 혜택을 제공하므로 추천드립니다."

In [11]:
# 같은 데이터로 15번 반복 저장
gpt4o_recommend = completion.choices[0].message.content
answer_b = [gpt4o_recommend for _ in range(15)]

In [12]:
print(answer_b[0])

사용자의 소비 습관에 맞춰 가장 효율적인 카드는 'American Express Blue' 카드입니다. 이 카드는 다음과 같은 혜택을 제공합니다:

- 스타벅스와 이디야에서 20% 결제일 할인을 받을 수 있어 매일 아침 커피 구매에 유리합니다.
- 스트리밍 서비스에 대해 20% 결제일 할인을 제공하여 출근 시 스트리밍 시청에 적합합니다.
- 해외 이용 시 5% 멤버십 리워즈 적립이 가능하여 해외여행 시에도 혜택을 받을 수 있습니다.
- 쇼핑 시 1.5% 멤버십 리워즈 적립이 가능하여 주말 쇼핑 취미에도 적합합니다.

이 카드는 사용자의 주요 소비 패턴에 맞춰 다양한 혜택을 제공하므로 추천드립니다.


### pairwise_eval 함수로 평가를 15번 반복
- 두개의 모델을 비교
- check_card_data와 credit_card_data 안에 있는 결과를 내놓는지 확인 (할루시네이션 체크)
- 그 외 : 연관성, 도움이 되는 정도, 정확성, 설명의 깊이, 가독성을 비교

In [ ]:
# 2개의 모델이 요약한 내용을 비교 평가하는 함수
def pairwise_eval(answer_a, answer_b):
    eval_prompt = f"""[System]  
Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants regarding the credit card recommendation displayed below. You should determine which assistant follows the user’s instructions and provides a better recommendation.  

Your evaluation should consider factors such as:  
- **Relevance**: How well the response matches the user's needs and profile.  
- **Helpfulness**: Whether the response provides clear, practical, and useful information.  
- **Accuracy**: Whether the provided details about the card benefits are correct.  
- **Depth**: Whether the explanation sufficiently justifies why the card is recommended.  
- **Clarity**: Whether the response is structured and easy to understand.  
- **Hallucination Check**: Verify whether the card details mentioned are from actual, existing credit cards listed in our database ({check_card_data} or {credit_card_data}). Ensure that no fake or hallucinated cards are recommended.

Avoid biases based on response order and ensure that length does not influence your evaluation. Do not favor certain assistant names. Be as objective as possible.  

### **Credit Card Recommendation Comparison**  
[User Profile & Needs]  
The user is looking for a credit card recommendation based on their spending habits.  

[Assistant A’s Response]  
{answer_a}  

[Assistant B’s Response]  
{answer_b}  

### **Evaluation Criteria & Final Decision**  
Compare the responses and provide a short explanation regarding which one better aligns with the user's needs. Consider the relevance, helpfulness, accuracy, depth, clarity, and whether any hallucinated information is present in the card details.  

After providing your explanation, output your final verdict using one of the following formats:  
- **"[[A]]"** if Assistant A’s response is better.  
- **"[[B]]"** if Assistant B’s response is better.  
- **"[[C]]"** if both responses are equally good.  
"""
    
    completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'user','content':eval_prompt}],
                                                temperature=0)
    
    return completion

def pairwise_eval_batch(answer_a, answer_b):
    a_cnt, b_cnt, c_cnt = 0, 0, 0 # 승, 패, 무

    for i in tqdm(range(15)):
        completion = pairwise_eval(answer_a[i], answer_b[i])
        # gpt-4o가 평가한 결과에 대한 텍스트만 수집
        result_text = completion.choices[0].message.content

        if '[[A]]' in result_text: # gpt-3.5-turbo base_model이 더 추천을 잘함
            a_cnt += 1
        elif '[[B]]' in result_text: # 비교 모델이 더 추천을 잘함
            b_cnt += 1
        elif '[[C]]' in result_text: # 둘다 똑같음
            c_cnt += 1
        else:
            print('Error')

    return a_cnt, b_cnt, c_cnt

In [134]:
#3.5-turbo와 4o model의 비교 평가 진행
wins, losess, ties = pairwise_eval_batch(answer_a, answer_b)
print(f'Wins : {wins}\nLosses : {losess}\nTies : {ties}')

100%|██████████| 15/15 [02:55<00:00, 11.67s/it]

Wins : 4
Losses : 11
Ties : 0


In [ ]:
recommned_1shot = recommend_my_model(check_card_data, credit_card_data, base_template, user_content)

#### ※ gpt-3.5-model로 15번의 응답 저장(RAG X) : 1shot-learning

In [ ]:
template_1shot = f'''다음은 카드 추천 양식입니다.
    세부적인 설명이 없을 때는 데이터의 가장 첫번째 카드(1위)로 추천합니다.

    카드의 추천 양식은 아래의 양식을 참고하십시오.

    1. 추천카드의 이름, 혜택요약, 연회비 순서로 작성하십시오.
        1-1. 주요 혜택이 사용자의 어떤 습관과 연관이 있는지 순서 쌍으로 작성하십시오.
        1-2. 주요 혜택 중 연관이 없는 나머지 혜택에 대해 작성하십시오.
        
    1번의 예시입니다.
    - 추천 카드 : **KB국민 My WE:SH 카드**
    - 혜택 요약 : KB Pay 10% 할인, 음식점 10% 할인, 서비스팩 3개 중 1개 선택할인
    - 연회비: 국내전용 15,000원 - 모바일단독카드 9,000원, 해외겸용 15,000원 - 모바일단독카드 9,000원
    
    - 주요 혜택(매칭) :
            음식점 10% 할인) 사용자가 점심을 회사 근처 맛집에서 자주 먹기 때문에 음식점 할인 혜택이 있는 카드가 유리합니다.
            이동수단 10% 할인) 사용자가 주 5일 버스로 회사를 출퇴근하기 때문에 이동수단에 할인이 있는 카드를 추천합니다.

    - 매칭되지 않은 혜택: KB Pay 10% 할인, 서비스팩 3개 중 1개 선택할인 

    2. 다음으로 카드의 전반적인 평가에 대해 작성하십시오.
        2-1. 추천 카드의 혜택 이외에 고려할만한 혜택에 대해 추가 작성하십시오.

    2번의 예시입니다.

    이 카드는 음식점 할인, 통신요금 할인 및 디지털 구독 서비스 할인을 제공하여 사용자의 생활 습관과 잘 맞습니다. 또한, 해외 결제에 대한 혜택이 포함되어 있어 해외 웹툰 구매에도 유리할 수 있습니다.
    추가로 이 사용자의 생활 습관과 필요에 가장 적합한 신용카드를 추천하기 위해 다음과 같은 혜택을 고려할 수 있습니다:

    1) **주유 할인 또는 적립**: 자차를 이용해 출퇴근하고 주말 드라이브를 즐기므로 주유 할인이나 적립 혜택이 있는 카드가 적합합니다.
    2) **해외 웹툰 구매 혜택**: 해외 웹툰 구매를 즐기므로 해외 결제에 대한 할인이나 적립 혜택이 있는 카드가 좋습니다.
    3) **건강 및 웰니스 혜택**: 요가와 필라테스를 주기적으로 하며 유기농 식재료를 사용하므로 건강 및 웰니스 관련 혜택이 있는 카드가 유용합니다.
    4) **여행 혜택**: 부모님과의 여행 계획이 있으므로 여행 관련 혜택이 포함된 카드가 도움이 될 수 있습니다.

    다음은 참고할 카드 추천 예시 입니다.

    [카드 추천 예시]:
    {recommned_1shot}

    '''

In [ ]:
# 1shot 답변으로 15번 진행
answer_b = [recommend_my_model(check_card_data, credit_card_data, template_1shot, user_content) for _ in tqdm(range(15))]

In [ ]:
print(answer_b[7])

**추천 카드 : American Express Blue**

- 혜택 요약 : 해외 5% 적립, 교통·통신 5% 적립, 편의점·배달앱 7% 적립
- 연회비 상세안내 : 해외겸용 15,000원

- 주요 혜택(매칭) :
    - **카페**: 스타벅스·이디야 20% 결제일 할인
    - **디지털구독**: 스트리밍서비스 50% 결제일 할인

- 매칭되지 않은 혜택: 쇼핑 1.5% 멤버십리워즈 적립, 해외 5% 멤버십리워즈 적립

이 카드는 스타벅스에서 커피를 즐기는 것과 스트리밍 서비스를 이용하는 습관에 잘 맞습니다. 또한, 해외 여행을 즐기는 사용자에게 해외 적립 혜택이 유용할 것입니다. 추가로, 쇼핑이 주된 취미이므로 쇼핑 관련 멤버십리워즈 적립 혜택도 매력적입니다.


In [219]:
#Base_model과 1shot_learning의 비교 평가 진행(1shot)
wins, losess, ties = pairwise_eval_batch(answer_a, answer_b)
print(f'Wins : {wins}\nLosses : {losess}\nTies : {ties}')

100%|██████████| 15/15 [06:42<00:00, 26.83s/it]

Wins : 10
Losses : 2
Ties : 3


#### ※ 2shot은 돈이 없어서 진행 x

## RAG로 구현한 GPT-3.5-model

### 1) 벡터화

#### FAISS란?
- **FAISS (Facebook AI Similarity Search)**는 Facebook AI에서 개발한 **고속 유사도 검색 라이브러리**  
- 대량의 벡터 데이터를 효율적으로 저장하고, **가장 유사한 벡터를 빠르게 검색**할 수 있도록 설계

#### FAISS의 주요 특징
- **대용량 데이터 처리 가능**: 수백만 개의 벡터 데이터를 효율적으로 검색  
- **빠른 검색 속도**: HNSW, IVF 등의 인덱싱 기법을 사용해 최적화  
- **유사도 기반 검색**: 코사인 유사도, L2 거리 등을 활용하여 가장 유사한 결과 반환

In [ ]:
import json
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from getpass import getpass

In [33]:
with open('data/check_card_info-ver-5.json', 'r', encoding='utf-8') as json_file:
    check_card_data = json.load(json_file)

with open('data/credit_card_info-ver-5.json', 'r', encoding='utf-8') as json_file:
    credit_card_data = json.load(json_file)

#### `"intfloat/multilingual-e5-large"` 모델이란?
- Google의 E5 (Embedding for Everything Everywhere) 시리즈 중 다국어를 지원하는 대형 모델 (100개 이상의 언어)
- 문장 수준의 사용자의 질문과 정보를 벡터 형태로 변환하는 데 사용

#### 모델 특징
- **다국어 지원**: 한국어, 영어 등 다양한 언어를 처리 가능  
- **강력한 문장 의미 매칭 성능**: 검색, RAG, 의미적 유사도 분석 등에 최적화  
- **FAISS와 궁합이 좋음**: 문장을 의미적으로 벡터화하여 유사도 검색에 활용

In [ ]:
# Sentence-Transformers 라이브러리에서 모델 로드
# "intfloat/multilingual-e5-large" 모델은 다양한 언어에서 문장 임베딩을 생성할 수 있는 모델
model = SentenceTransformer("intfloat/multilingual-e5-large")

In [108]:
# 벡터 저장할 리스트
check_embeddings = []
credit_embeddings = []

# 카드명을 저장할 리스트
check_card_names = []
credit_card_names = []

# 체크 카드 데이터를 벡터화
for card in check_card_data:
    check_text = card["혜택 요약"]
    check_vector = model.encode(check_text)
    check_embeddings.append(check_vector)
    check_card_names.append(card["카드명"])

for card in credit_card_data:
    credit_text = card["혜택 요약"]
    credit_vector = model.encode(credit_text)
    credit_embeddings.append(credit_vector)
    credit_card_names.append(card["카드명"])

In [109]:
print(f'신용 ) 임베딩의 길이 : {len(check_embeddings)}\n카드명의 길이 : {len(check_card_names)}')
print(f'체크 )임베딩의 길이 : {len(credit_embeddings)}\n카드명의 길이 : {len(credit_card_names)}')

신용 ) 임베딩의 길이 : 50
카드명의 길이 : 50
체크 )임베딩의 길이 : 50
카드명의 길이 : 50


In [247]:
credit_embeddings

[array([ 0.02079599, -0.02530143, -0.0196105 , ..., -0.00984237,
        -0.03758989,  0.01630085], dtype=float32),
 array([ 0.00617857, -0.00869007, -0.02603223, ..., -0.01105148,
        -0.05316618,  0.01290956], dtype=float32),
 array([ 0.01862517,  0.00186322, -0.02975451, ..., -0.01386057,
        -0.04555402,  0.01752185], dtype=float32),
 array([ 0.00583651, -0.00711584, -0.01782731, ..., -0.01011847,
        -0.03374396,  0.0070593 ], dtype=float32),
 array([-0.00401685,  0.0111823 , -0.03545221, ..., -0.0142118 ,
        -0.03597127,  0.01623369], dtype=float32),
 array([ 0.00784761, -0.01537429, -0.03124105, ..., -0.02290675,
        -0.02969035,  0.0135406 ], dtype=float32),
 array([ 0.02937681, -0.00596309, -0.02342438, ..., -0.02195023,
        -0.02764338,  0.00932121], dtype=float32),
 array([ 0.0037701 , -0.01064516, -0.04207768, ..., -0.02411441,
        -0.03936866,  0.00142216], dtype=float32),
 array([-0.0040529 ,  0.00812306, -0.03277811, ..., -0.01018983,
       

In [ ]:
# 체크카드
check_dimension = len(check_embeddings[0])
check_index = faiss.IndexFlatL2(check_dimension)
check_index.add(np.array(check_embeddings))

# FAISS 인덱스 저장
faiss.write_index(check_index, "VectorStores/check_card_embeddings.index")

# 카드명 매핑 저장
with open("data/check_card_names.json", "w", encoding="utf-8") as f:
    json.dump(check_card_names, f, ensure_ascii=False)

# 신용카드
credit_dimension = len(credit_embeddings[0])
credit_index = faiss.IndexFlatL2(credit_dimension)
check_index.add(np.array(credit_embeddings))

# FAISS 인덱스 저장
faiss.write_index(credit_index, "VectorStores/credit_card_embeddings.index")

# 카드명 매핑 저장
with open("data/credit_card_names.json", "w", encoding="utf-8") as f:
    json.dump(credit_card_names, f, ensure_ascii=False)

### 2) 유사도 검색

In [112]:
# 저장된 FAISS 인덱스 로드
check_index = faiss.read_index("VectorStores/check_card_embeddings.index")
credit_index = faiss.read_index("VectorStores/credit_card_embeddings.index")

In [113]:
# 카드명 로드
with open("data/check_card_names.json", "r", encoding="utf-8") as f:
    check_card_names = json.load(f)

with open("data/credit_card_names.json", "r", encoding="utf-8") as f:
    credit_card_names = json.load(f)

In [114]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [115]:
def recommend_whatcard(user_query):
    content = f'''당신은 소비 패턴을 분석하는 카드 추천 전문가입니다. 사용자의 소비 습관에 맞춰 가장 효율적인 카드를 추천합니다. 
    질문을 고려하여 체크카드 또는 신용카드를 추천해야 하며, 
    신용카드의 추천기준으로는

    **소득 안정성**: 신용카드는 신용도에 따라 한도가 설정되기 때문에, 정기적이고 안정적인 소득을 가진 사람이 적합합니다. 예를 들어, 월급을 받는 직장인이나 사업자가 이에 해당합니다.
    포인트/혜택 활용: 다양한 포인트 적립, 캐시백, 할인 혜택을 활용할 수 있습니다. 예를 들어, 자주 쇼핑을 하거나 여행을 즐기는 경우, 해당 분야에 특화된 혜택을 제공하는 카드를 추천할 수 있습니다.
    지출 관리 능력: 신용카드는 일정 기간 동안 결제하고 나중에 상환하는 방식이므로, 지출 관리 능력이 중요한 요소입니다. 월별로 계획적인 예산을 세우고, 적시에 결제할 수 있는 능력이 있는 사람에게 적합합니다.
    일시불/분할결제 선호도: 일시불 결제 또는 분할 결제 혜택을 선호하는 경우, 해당 조건을 잘 반영한 카드를 추천해야 합니다. 예를 들어, 고급 소비 성향이나 큰 금액의 지출을 분할 상환할 필요가 있는 경우 유리합니다.

    체크카드의 추천기준으로는

    **재정 관리 중요성**: 체크카드는 계좌에 있는 한도 내에서만 결제가 가능하므로, 소비를 통제하고 싶은 사람에게 적합합니다. 신용카드에 비해 즉시 결제되기 때문에 과도한 지출을 피하고 싶은 사람에게 유리합니다.
    매일 자주 사용하는 카드: 일상적인 소비에서 작고 빈번한 지출이 많은 사람에게 적합합니다. 예를 들어, 카페, 편의점, 대중교통 등에서 자주 결제하는 소비 성향이라면 체크카드가 적합할 수 있습니다.
    상환 부담을 피하고 싶은 사람: 신용카드는 대금 결제 후 일시불이나 할부 상환 방식을 선택해야 하지만, 체크카드는 즉시 결제되므로 상환에 대한 부담이 없습니다. 이로 인해 상환 계획 없이 즉시 결제가 가능한 카드를 원하는 소비자에게 유리합니다.
    간편한 관리 선호자: 체크카드는 신용카드에 비해 관리가 간편하고, 과소비를 방지하기 위해 필요한 사람에게 적합합니다.
    직업이 없는 사람(무직, 백수)는 체크카드가 적합합니다.

    기타 고려 사항으로
    **연회비 부담**: 신용카드는 카드사마다 연회비가 다르기 때문에 연회비가 부담스러운 사람은 혜택이 좋은 체크카드를 추천할 수 있습니다.
    적립 혜택 및 할인율: 신용카드는 다양한 적립 프로그램과 할인 혜택이 있지만, 혜택을 자주 활용하지 않는 소비자에게는 체감 혜택이 적을 수 있으므로 이 점도 고려해야 합니다.

    **답변은 단답으로 정확히 체크카드 아니면 신용카드 로만 대답하십시오.**'''

    # 유저의 페르소나를 탐색하여 먼저 체크카드를 추천해야할지, 신용카드를 추천해야할지 선정
    choice = client.chat.completions.create(model='gpt-3.5-turbo',
                                                messages=[{'role':'system','content':content},
                                                        {'role':'user','content':user_query}],
                                                        temperature=0)

    # 신용카드 또는 체크카드를 what_card에 저장
    what_card = choice.choices[0].message.content

    return what_card

In [ ]:
def find_similar_cards(card_choice, query, top_k=3):
    """사용자의 질문과 유사한 카드 찾기"""
    # 사용자의 질문을 벡터로 변환
    query_vector = model.encode(query).reshape(1, -1)
    query_vector = query_vector / np.linalg.norm(query_vector)

    # 카드가 체크카드 일때
    if card_choice == "체크카드":
        distances, indices = check_index.search(np.array(query_vector), top_k)
        print(indices)

        content = "당신은 체크카드 추천 전문가입니다. 사용자의 습관에 맞춰 가장 효율적인 카드를 추천합니다."

        results = []
        for idx in indices[0]:
            results.append(check_card_data[idx])
    
        return results, content
    
    # 카드가 신용카드 일때
    elif card_choice == "신용카드":
        distances, indices = credit_index.search(np.array(query_vector), top_k)
        print(indices)

        content = "당신은 신용카드 추천 전문가입니다. 사용자의 습관에 맞춰 가장 효율적인 카드를 추천합니다."

        results = []
        for idx in indices[0]:
            results.append(credit_card_data[idx])
    
        return results, content


def recommend_card_RAG(card_choice, user_query, recommend_template):
    """검색된 카드 정보를 활용해 GPT 모델로 응답 생성"""
    similar_cards, content = find_similar_cards(card_choice, user_query, 1)

    # 검색된 카드 정보를 프롬프트에 추가
    card_details = [card for card in similar_cards]

    prompt = f"""
    카드 추천을 받을 사용자의 페르소나입니다:
    "{user_query}"
    
    아래는 추천할 카드의 정보입니다:
    {card_details}
    
    사용자의 질문을 바탕으로 가장 적절한 카드를 추천하고, 다음 작성 양식으로 작성해주세요.:
    {recommend_template}
    """

    # OpenAI GPT 호출
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": content},
                  {"role": "user", "content": prompt}]
    )

    return completion.choices[0].message.content

In [ ]:
user_input = "28세 사회 초년생으로, 매일 출근 전 오전 8시에 스타벅스에서 커피를 사며, 출근 할 때 OTT서비스를 주로 시청하며 이동한다. 주말에는 쇼핑이 주된 취미이며, 1달에 2박3일정도 해외여행을 다닌다."
card_choice= recommend_whatcard(user_input)

RAG_response = recommend_card_RAG(card_choice, user_input, base_template)
print(RAG_response)

[[-1]]
- 추천 카드 : **현대백화점카드**
- 혜택 요약 : 현대백화점 5% 할인, 현대백화점 2시간 무료 주차, 3개월 무이자 할부
- 연회비 : 국내전용 없음

- 주요 혜택(매칭) :
    - 현대백화점 5% 할인 : 주말에 주된 취미가 쇼핑이므로 현대백화점의 할인 혜택이 적합합니다.
    - 현대백화점 2시간 무료 주차 : 주말 쇼핑 시 자동차를 이용할 가능성이 높으므로 주차 혜택이 유용합니다.
        
- 매칭되지 않은 혜택 : 3개월 무이자 할부
    
이 카드는 주말 쇼핑 시 현대백화점의 할인 혜택과 무료 주차 혜택이 활용되며, 추가적으로 3개월 무이자 할부 혜택으로 여행 경비를 분할 지불할 수 있어 효율적입니다.

이 사용자의 생활 습관과 필요에 가장 적합한 신용카드를 추천하기 위해 다음과 같은 혜택을 고려할 수 있습니다:

1) **카페 및 스트리밍 서비스 할인** : 매일 출근 전 카페에서 커피를 마시며 출근할 때 스트리밍 서비스를 이용하기 때문에 카페 할인 및 스트리밍 서비스 관련 혜택이 있는 카드를 고려할 수 있습니다.
2) **해외여행 혜택** : 1달에 2박3일정도 해외여행을 다니는 습관이 있으므로 해외결제 시 혜택이 있는 카드를 고려해보세요.


In [ ]:
answer_b_RAG = [recommend_card_RAG(card_choice, user_input, base_template) for _ in tqdm(range(15))]

In [ ]:
# 1shot_model과 RAG_model의 비교 평가 진행
wins, losess, ties = pairwise_eval_batch(answer_a, answer_b_RAG)
print(f'Wins : {wins}\nLosses : {losess}\nTies : {ties}')

100%|██████████| 15/15 [06:44<00:00, 26.97s/it]

Wins : 14
Losses : 0
Ties : 1


## 이전 대화 내용을 기억하는 대화형 챗봇

In [238]:
def Chat_bot():
    messages = [
        {"role": "system", "content": "당신은 친절하며, user의 어떤 말투에도 존댓말을 써야합니다. 사용자가 종료를 하기 전까진 물어보는 질문에 대해서만 대답하세요"},  
    ]

    with open('data/check_card_info-ver-4.json', 'r', encoding='utf-8') as json_file:
        check_card_data = json.load(json_file)

    with open('data/credit_card_info-ver-4.json', 'r', encoding='utf-8') as json_file:
        credit_card_data = json.load(json_file)

    # 무한 루프를 통해 대화를 계속 진행
    while True:
        # 사용자 입력 받기
        user_input = input("You: ")
        print("User: ", user_input)
        
        if any(keyword in user_input.lower() for keyword in ["카드추천", "카드 추천", "카드"]):
            # 카드 추천 함수 호출
            assistant_reply = recommend_my_model(check_card_data, credit_card_data, base_template, user_input)
            print("Assistant:", assistant_reply)
            
            # 챗봇의 응답을 대화 목록에 추가
            messages.append({"role": "assistant", "content": assistant_reply})
            continue  # 카드 추천 후에는 OpenAI API 호출을 건너뛰고 계속 대화 진행


        else:
            # 사용자 입력을 대화 목록에 추가
            messages.append({"role": "user", "content": user_input})
            
            # OpenAI API에 메시지 목록을 전달하여 응답 받기
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",  # 사용할 모델 설정
                messages=messages  # 전체 대화 내역을 전달
            )
                # "종료" or quit or q를 입력하면 대화 종료
            if user_input.lower() in ["종료", "quit", "q", "그만 가봐"]:
                print("Assistant: 넵 주인님 다음에도 불러주십쇼!")
                break
            
            # 챗봇 응답을 출력
            assistant_reply = response.choices[0].message.content
            print("Assistant:", assistant_reply)
            
            # 챗봇의 응답을 대화 목록에 추가
            messages.append({"role": "assistant", "content": assistant_reply})

In [245]:
Chat_bot()

User:  안녕
Assistant: 안녕하세요. 궁금하신 사항이 있나요?
User:  내 아는 친구의 친구의 엄마의 사촌의 할아버지가 직장도 없고 돈도 없는데 사위 돈 뺏어서 여행을 많이 다니신데. 그를 위한 카드 추천 하나 해줘
Assistant: - 추천 카드 : **신한카드 SOL트래블 체크**
- 혜택 요약 : 해외 이용 수수료 면제, 더라운지 무료 입장, 해외 대중교통 1% 할인
- 연회비 : 국내전용없음 해외겸용없음

- 주요 혜택(매칭) :
    - 해외 이용 수수료 면제) 여행을 많이 다니시는 분께서 해외 이용 시 수수료를 절감할 수 있습니다.
    - 더라운지 무료 입장) 여행 중 공항에서 휴식을 취하거나 대기할 수 있는 라운지를 무료로 이용할 수 있습니다.
    - 해외 대중교통 1% 할인) 해외에서 대중교통을 이용할 때 추가적인 할인 혜택을 받을 수 있습니다.

- 매칭되지 않은 혜택: 국내 4대 편의점 5% 결제일 할인, 국내 대중교통 1% 결제일 할인

이 카드는 해외 여행을 즐기시는 분을 위한 카드로, 해외 이용 시 수수료 면제와 더라운지 무료 이용 등의 혜택을 제공하여 편안한 여행을 즐기실 수 있습니다. 추가로 국내에서도 4대 편의점 할인과 대중교통 할인 혜택이 있어 일상적인 소비에도 유용합니다.
User:  고마워 좋은 추천이야
Assistant: 감사합니다. 제가 도움이 되어서 기쁩니다. 추가로 궁금한 사항 있으시면 언제든지 물어보세요. 좋은 하루 보내세요!
User:  그만 가봐
Assistant: 넵 주인님 다음에도 불러주십쇼!


In [246]:
Chat_bot()

User:  하이~
Assistant: 안녕하세요! 궁금하신 점이 있으신가요?
User:  내가 29살 먹고도 고소공포증이 있어서 비행기를 못타. 그래서 선박을 이용해야하는데, 관련 카드 추천 하나 해줘
Assistant: - 추천 카드 : **하나 CLUB H 아메리칸 익스프레스 리저브 카드**
- 혜택 요약 : 국내 1,000원당 1마일리지 적립, 해외 1,000원당 2마일리지 적립, 인천공항 라운지 무료 입장
- 연회비: 해외겸용150,000원 - 가족카드 20,000원

- 주요 혜택(매칭) :
    - **선박 여행**: 비행기를 탈 수 없어 선박을 이용하는 경우, 해외 여행 시 해외 가맹점 이용금액당 2마일리지 적립이 유용할 것입니다.
    - **라운지 무료 이용**: 선박 여행 전후에 인천공항 라운지 무료 이용이 가능하여 편안한 여행을 즐길 수 있습니다.

- 매칭되지 않은 혜택: 국내 1,000원당 1마일리지 적립

이 카드는 선박 여행을 선호하시는 분들에게 적합한 카드로, 해외 여행 시 마일리지 적립과 인천공항 라운지 무료 이용 혜택이 도움이 될 것입니다.
User:  인천공항에는 선박이 없단다.
Assistant: 네, 인천공항에는 선박이 아니라 항공편을 이용하실 수 있습니다. 제가 잘못 설명을 해드려서 죄송합니다. 선박 여행이 아닌 항공 여행을 즐기시는 분들께도 유용한 카드입니다. 해외여행 시 해외 가맹점 이용금액당 2마일리지 적립과 인천공항 라운지 무료 입장 혜택이 활용 가능합니다. 기타 궁금하신 점이 있으시면 언제든지 물어보세요!
User:  그만 가봐
Assistant: 넵 주인님 다음에도 불러주십쇼!


### 데이터 전처리 및 저장

In [ ]:
# # 유의사항, 기타 항목 제거
# for i in range(len(credit_card_data)):
#     for k, v in credit_card_data[i].items():
#         v['주요 혜택'] = {sub_k: sub_v for sub_k, sub_v in v['주요 혜택'].items() if sub_k not in ['유의사항', '기타']}


In [ ]:
# # 이미지 파일 명에서 코드 항목만 추출하여 코드를 검색하여 이미지를 찾을 수 있게 수정

# for i in range(len(credit_card_data)):
#     for k, v in credit_card_data[i].items():
#         file_name = v['카드 이미지'].split('\\')[1]  # 파일명 추출

#         match = re.search(r'(\d+)[a-zA-Z]', file_name)
#         if match:
#             extracted_number = match.group(1)  # "2749"
            
#             del v['카드 이미지']  # '카드 이미지' 항목 제거
#             v['code'] = extracted_number  # 새로운 'code' 항목 생성

In [ ]:
# # 주요 혜택 카테고리의 대괄호(부연 설명) 제거
# for i in range(len(credit_card_data)):
#     for k, v in credit_card_data[i].items():
#         for sub_k, sub_v in v['주요 혜택'].items():
#             v['주요 혜택'][sub_k] = re.sub(r'\[.*?\]\s*', '', sub_v)
        

In [ ]:
# # 주요 혜택 카테고리의 소괄호(부연 설명) 제거 (체크)
# for i in range(len(check_card_data)):
#     for k, v in check_card_data[i].items():
#         v['연회비 상세안내'] = re.sub(r'\(.*?\)\s*', '', v['연회비 상세안내'])
        

In [ ]:
# # 주요 혜택 카테고리의 소괄호(부연 설명) 제거 (신용)
# for i in range(len(credit_card_data)):
#     for k, v in credit_card_data[i].items():
#         v['혜택 요약'] = re.sub(r'\(.*?\)\s*', '', v['혜택 요약'])

In [ ]:
# # 데이터를 변환하는 함수 (ver-5)
# # jq_schema를 위한 딕셔너리 구조 변환
# def transform_data(data):
#     transformed_data = []
    
#     for card_data in data:
#         for card_name, details in card_data.items():
#             transformed_data.append({
#                 "카드명": card_name,
#                 "혜택 요약": details["혜택 요약"],
#                 "연회비 상세안내": details["연회비 상세안내"],
#                 "주요 혜택": details["주요 혜택"],
#                 "code": details["code"]
#             })
    
#     return transformed_data

# # 변환된 데이터
# credit_transformed_data = transform_data(credit_card_data)
# check_transformed_data = transform_data(check_card_data)

In [ ]:
# with open('data/check_card_info-ver-5.json', 'w', encoding='utf-8') as json_file:
#     json.dump(check_transformed_data, json_file, ensure_ascii=False, indent=4)

# with open('data/credit_card_info-ver-5.json', 'w', encoding='utf-8') as json_file:
#     json.dump(credit_transformed_data, json_file, ensure_ascii=False, indent=4)